# 🤖 03 – Modelling & Evaluation

**Objective:**  
Train, evaluate, and interpret a baseline machine learning model to predict whether a Netflix title is a potential “hit” based on its metadata features.

**Scope of this notebook:**
- Load the processed dataset from **`netflix_clean`** (created in Notebook 02)
- Select relevant numeric and categorical features for modelling  
- Split the dataset into training and test sets  
- Build and train a baseline classification model (Random Forest or Logistic Regression)  
- Evaluate model performance using key metrics:
  - Accuracy, Precision, Recall, F1-score, ROC-AUC  
- Log model parameters and metrics using **MLflow** for experiment tracking  
- Analyse feature importance and interpretability  
- Save the trained model for downstream use (e.g., predictions or dashboard integration)

**Outcome:**  
A reproducible, MLflow-tracked baseline model capable of classifying titles as *Hit* or *Non-Hit*, with performance metrics and feature insights ready for visualisation and reporting in Notebook 04.